In [2]:
'''libraries list with help showing the version of the libraries used in this nodebook'''
libraries = []
import random
'''Datasets, arrays and json files'''

import pandas as pd
import numpy as np
import pickle

libraries.append('pandas')
libraries.append('numpy')
libraries.append('pickle')

'''Following progress'''
from tqdm.notebook import tqdm
libraries.append('tqdm')

'''Utilities'''
import collections
from pathlib import Path
from datetime import datetime
import os
#import powerlaw 
libraries.append('collections')
libraries.append('pathlib')
libraries.append('datetime')
libraries.append('os')
#libraries.append('powerlaw')

'''Plots and figures'''
%matplotlib notebook

import matplotlib.pyplot as plt
#import plotly.express as px
libraries.append('matplotlib')
#libraries.append('plotly')

plt.rcParams["figure.facecolor"] = "w"

'''Pthon and library version'''
import types
import pkg_resources
import sys
from platform import python_version

In [2]:
'''To display version of Software being used'''
print('Version of python installed: {}' .format(sys.version))
print('Version of python being used: {}' .format(python_version()))
print('\nNon-built in libraries being used:')

for m in pkg_resources.working_set:
    if m.project_name.lower() in libraries:
        print('{}, version {}'.format(m.project_name,m.version))

Version of python installed: 3.8.10 (default, May 19 2021, 11:01:55) 
[Clang 10.0.0 ]
Version of python being used: 3.8.10

Non-built in libraries being used:
tqdm, version 4.62.3
pandas, version 1.3.3
numpy, version 1.19.2
matplotlib, version 3.4.3


In [3]:
def to_check_duplicates(df, col):
    '''Helper function to check if there is duplicated in a DataFrame df, on a defined
    column col. It will return the duplicated rows, if there is any.'''
    ids = df[col]
    duplicated = df[ids.isin(ids[ids.duplicated()])]
    return duplicated

def print_file_names():
    '''Helper function to search for files'''
    cwd = os.getcwd()  # Get the current working directory (cwd)
    files = os.listdir(cwd)  # Get all the files in that directory
    print("Files in %r: %s" % (cwd, files))

In [5]:
'''Filepath and folders that are going to be used'''
filepath = '../../data/covid19/downloaded_data/'
folder = ['Brazil/','Chile/','NSW/','USA']

In [6]:
saving_path = '../_generated_files/'
!ls {saving_path}

AllScales24Oct.pkl            postcodeCases24Oct.csv
Brazil24Oct.csv               postcodeCaseswDates24Oct.csv
Brazil_by_dates               results_covid19_NSW.pickle
Chile_26Oct.csv               results_covid19_USA.pickle
Chile_wdates23Sept.csv        results_covid19_brazil.pickle
NSW_SUA_SA2_LGA24Oct.pkl      results_covid19_chile.pickle
USA14Oct.csv                  x_covid19_NSW.pickle
USA14Octstate.csv             x_covid19_USA.pickle
covid19_NSW.csv               x_covid19_brazil.pickle
covid19_brazil.csv            x_covid19_chile.pickle
covid19_chile.csv             y_covid19_NSW.pickle
covid_USA.csv                 y_covid19_USA.pickle
dataframe_empty_USA.pikle     y_covid19_brazil.pickle
dataframe_empty_brazil.pikle  y_covid19_chile.pickle
list_delays_Brazil.pikle


In [43]:
!ls {filepath+folder[-1]}

co-est2020.csv  us-counties.csv


# NSW
The data contains number of cases by Postalcode and LGA, including notification dates.
One row explain one case location and date of ocurrence.

The aim is to find information about COVID-19 cases with information of a good representation of functional pupulation centre. Significant Urban Area (SUA) seems to be a suitable definition with these purpose in mind. 

The data will be preprocesses and the concordance with SUA will be found. If there is no SUA that can be linked with PostalCode, a smaller administrative definition (SA2) will be use. In case that no SA2 can be paired with Postalcode either, LGA will be used.

In [58]:
''' Keys: notification_date postcode lhd_2010_code lhd_2010_name lga_code19 lga_name19 '''
df = pd.read_csv(filepath+folder[2]+'data202126OctNSW.csv')

* Latest data (downloaded October 24th): 73,596 total cases in NSW.
* 70,202 total cases downloaded at 14th of October.
* 54,730 total cases downloaded at 23th of September.
* 46,818 total cases downloaded at 16th of September.

In [59]:
df['notification_date'].count() 

73596

The last cases recorded in the file are from 24th of October.

In [60]:
df['notification_date'].tail() 

73591    2021-10-24
73592    2021-10-24
73593    2021-10-24
73594    2021-10-24
73595    2021-10-24
Name: notification_date, dtype: object

To check for null values. There are no null values in the file.

In [61]:
df['postcode'].value_counts()
df.loc[pd.isnull(df.postcode)] 

,notification_date,postcode,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19


There are 1,081 values without information about location. 
The values with 'postcode' == 'None' will be dropped, because it is not possible to assign them to a Significant Urban Area in a straight forward manner.

There are 980 values without NA lga_name value. 

In [62]:
none_postcode = df[df['postcode'] == 'None']
print(none_postcode.count())

df[df['postcode'].isna()].count()
print(df.isna().value_counts())

notification_date    1081
postcode             1081
lhd_2010_code         166
lhd_2010_name         166
lga_code19            166
lga_name19            166
dtype: int64
notification_date  postcode  lhd_2010_code  lhd_2010_name  lga_code19  lga_name19
False              False     False          False          False       False         72616
                             True           True           True        True            980
dtype: int64


In [63]:
df = df[df['postcode'] != 'None']
'''Summary of NA values remaining in the dataframe'''
df.isna().value_counts()

notification_date  postcode  lhd_2010_code  lhd_2010_name  lga_code19  lga_name19
False              False     False          False          False       False         72450
                             True           True           True        True             65
dtype: int64

65 registers without lg_code19/lga_name19 are still on the DataFrame.

In [64]:
df[df['lga_name19'].isnull()]

,notification_date,postcode,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19
33,2020-03-07,2091,NaN,NaN,NaN,NaN
437,2020-03-20,2612,NaN,NaN,NaN,NaN
3006,2020-04-30,2612,NaN,NaN,NaN,NaN
3049,2020-05-13,2602,NaN,NaN,NaN,NaN
3096,2020-06-07,2606,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
48629,2021-09-16,2906,NaN,NaN,NaN,NaN
49708,2021-09-17,2902,NaN,NaN,NaN,NaN
63998,2021-10-02,2913,NaN,NaN,NaN,NaN
68055,2021-10-09,2615,NaN,NaN,NaN,NaN


To remove the columns with information that is not relevant for the following analysis, and change the 'notification_date' to a datetime format that will be more useful. 72,515 registers remain.

In [65]:
'''Drop unused columns'''
df = df.drop(columns =['lhd_2010_code', 'lhd_2010_name', 'lga_code19'])

'''Change 'notification_date' to time format'''
df['notification_date']= pd.to_datetime(df['notification_date'])
df['notification_date'].count()

72515

In [66]:
df.head()

,notification_date,postcode,lga_name19
0,2020-01-25,2134,Burwood (A)
1,2020-01-25,2121,Parramatta (C)
2,2020-01-25,2071,Ku-ring-gai (A)
3,2020-01-27,2033,Randwick (C)
4,2020-03-01,2163,Fairfield (C)


In [67]:
'''Checking datatypes'''
print(type(df['notification_date'][0])) #timestamp
print(type(df['postcode'][0])) #str

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>


Seven registers have a 'postcode' defined as 'Masked'. As it cannot be identified as aby Significat Urban Area

In [68]:
what = df[df['postcode'] == 'Masked']
what.count() #7 notifications are masked. I will drop them.

notification_date    7
postcode             7
lga_name19           7
dtype: int64

To remove the identified 'Masked' and 'None' values of postcode. 
Finally, 72,508 cases remain.

In [69]:
df = df[df['postcode'] != 'Masked'] #Removing 'Masked' postcodes
df = df[df['postcode'] != 'None'] #Removing 'None' postcodes
df['notification_date'].count()

72508

## Total cases per postcode

In [77]:
def postalcases_total(df, date):
    '''Stores total cases (the count) on a file and it returns the same DataFrame stored'''
    df['postcode'].value_counts().reset_index().to_csv(saving_path+'postcodeCases'+date+'.csv' , 
                                                       header=['postalcodes', 'cases'], 
                                                       index = False)
    '''Put label on the columns'''
    df_postalcases = pd.read_csv(saving_path+'postcodeCases'+ date +'.csv')
    df_postalcases.columns = ['postcode', 'cases']
                                 
    return df_postalcases

In [78]:
''' To generate a file with total cases'''
df_postalcases = postalcases_total(df, '24Oct')
df_postalcases.head()
df_postalcases['cases'].sum()

72508

## Preparing Dataset

Containing date, postcode, lga and total population.
SUA and SA2 will be added later.

In [79]:
df_wdates = df.copy()
df_wdates.to_csv(saving_path+'postcodeCaseswDates24Oct.csv' , header=['date', 'postcode', 'lga_name19'], 
                 index = False)

In [80]:
df_wdates.head()

,notification_date,postcode,lga_name19
0,2020-01-25,2134,Burwood (A)
1,2020-01-25,2121,Parramatta (C)
2,2020-01-25,2071,Ku-ring-gai (A)
3,2020-01-27,2033,Randwick (C)
4,2020-03-01,2163,Fairfield (C)


In [81]:
df_wdates.shape

(72508, 3)

to read the file 'postcode_population.csv' containing the population per postalcode.

In [82]:
df_postal = pd.read_csv(filepath+'NSW/postcode_population.csv') 
df_postal['postcode'] = df_postal['postcode'].astype(np.str)
type(df_postal['postcode'][0])

str

In [83]:
df_wdates['lga_name19'].value_counts()
df_wdates.loc[pd.isnull(df.lga_name19)] 
df_wdates.shape

(72508, 3)

In [84]:
df_wdates

,notification_date,postcode,lga_name19
0,2020-01-25,2134,Burwood (A)
1,2020-01-25,2121,Parramatta (C)
2,2020-01-25,2071,Ku-ring-gai (A)
3,2020-01-27,2033,Randwick (C)
4,2020-03-01,2163,Fairfield (C)
...,...,...,...
73591,2021-10-24,2620,Queanbeyan-Palerang Regional (A)
73592,2021-10-24,2170,Liverpool (C)
73593,2021-10-24,2250,Central Coast (C) (NSW)
73594,2021-10-24,2170,Liverpool (C)


In [85]:
df_postal

,postcode,population
0,2000,52601
1,2006,1828
2,2007,10023
3,2008,13373
4,2009,14748
...,...,...
613,2877,3849
614,2878,459
615,2879,696
616,2880,18659


In [86]:
merged = pd.merge(df_wdates, df_postal,on=['postcode'], how='outer') ####inner
merged = merged.reset_index(drop=True)
merged

,notification_date,postcode,lga_name19,population
0,2020-01-25,2134,Burwood (A),16593.0
1,2020-03-29,2134,Burwood (A),16593.0
2,2020-03-30,2134,Burwood (A),16593.0
3,2020-04-02,2134,Burwood (A),16593.0
4,2020-04-08,2134,Burwood (A),16593.0
...,...,...,...,...
72629,NaT,2876,NaN,420.0
72630,NaT,2877,NaN,3849.0
72631,NaT,2878,NaN,459.0
72632,NaT,2879,NaN,696.0


Merged DataFrame includes date, postcode, lga, total population (by_postcode)

In [87]:
merged = merged.rename({'population':'total_population'}, axis=1)
merged.sort_values(by=['postcode'])

,notification_date,postcode,lga_name19,total_population
30569,2021-09-06,2000,Sydney (C),52601.0
30558,2021-09-05,2000,Sydney (C),52601.0
30557,2021-09-04,2000,Sydney (C),52601.0
30556,2021-09-04,2000,Sydney (C),52601.0
30555,2021-09-04,2000,Sydney (C),52601.0
...,...,...,...,...
70558,2021-04-25,2914,NaN,NaN
70559,2021-04-25,2914,NaN,NaN
70560,2021-05-13,2914,NaN,NaN
70561,2021-08-20,2914,NaN,NaN


There are 65 cases without LGA information. Only one of them has population. 
As no population is found on almost all these LGAs, the data will be dropped.
127 rows have info about population and none cases (how many postcodes are duplicated?)

In [88]:
merged[merged['lga_name19'].isna()].count() 

notification_date     65
postcode             191
lga_name19             0
total_population     127
dtype: int64

In [89]:
merged.shape

(72634, 4)

There are 341 cases where postalcode has no population. 
The data will be dropped.

In [90]:
merged[merged['total_population'].isna()]

,notification_date,postcode,lga_name19,total_population
7276,2020-03-07,2091,NaN,NaN
29603,2020-03-18,2540,Shoalhaven (C),NaN
29604,2020-03-24,2540,Shoalhaven (C),NaN
29605,2020-03-27,2540,Shoalhaven (C),NaN
29606,2020-03-27,2540,Shoalhaven (C),NaN
...,...,...,...,...
71396,2021-05-19,2902,NaN,NaN
71397,2021-09-17,2902,NaN,NaN
71398,2021-06-22,2900,NaN,NaN
71425,2021-08-03,2899,NaN,NaN


In [91]:
print(merged[merged['postcode'] == '2540'].notification_date.count())
#Population of postcode 2540 Shoalhaven (C) https://en.wikipedia.org/wiki/City_of_Shoalhaven

mask = (merged.postcode == '2540')
merged['total_population'][mask] = 104371

108


/var/folders/2r/dxh5vfq12h7bhlg2f88kpl440000gn/T/ipykernel_24272/3148102510.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['total_population'][mask] = 104371


In [92]:
merged[merged['postcode'] == '2540'].lga_name19.count()

108

In [93]:
merged[merged['notification_date'].isna()].count() #0 cases

notification_date      0
postcode             126
lga_name19             0
total_population     126
dtype: int64

In [94]:
#https://en.wikipedia.org/wiki/Campbell,_Australian_Capital_Territory
#https://postcodes-australia.com/postcodes/2902
#Not part of NSW (Australian Capital Territory)

merged = merged[merged['postcode'] != '2612']
merged = merged[merged['postcode'] != '2902']

In [95]:
merged[merged['total_population'].isna()]

,notification_date,postcode,lga_name19,total_population
7276,2020-03-07,2091,NaN,NaN
44091,2020-03-21,2620,Queanbeyan-Palerang Regional (A),NaN
44092,2020-03-24,2620,Queanbeyan-Palerang Regional (A),NaN
44093,2020-03-27,2620,Queanbeyan-Palerang Regional (A),NaN
44094,2020-03-28,2620,Queanbeyan-Palerang Regional (A),NaN
...,...,...,...,...
71142,2021-10-09,2615,NaN,NaN
71143,2021-01-24,2601,NaN,NaN
71398,2021-06-22,2900,NaN,NaN
71425,2021-08-03,2899,NaN,NaN


In [96]:
merged = merged[merged.lga_name19.notnull()]
merged[merged['lga_name19'].isna()].count() #remove data without LGA

notification_date    0
postcode             0
lga_name19           0
total_population     0
dtype: int64

72,443 cases remain in the DataFrame. This is a 98.43% of the cases on the initial dataset.

In [97]:
merged['notification_date'].count()

72443

In [98]:
round(merged.shape[0]*100/73596,4)

98.4333

In [40]:
merged[merged.isna()].any() #Now the data has no null values.

notification_date    False
postcode             False
lga_name19           False
total_population     False
dtype: bool

## Add SUA and SA2 to NSW Data

In [99]:
SUA = pd.read_csv(filepath+"/NSW/SUA_2016_AUST.csv")
SUA.head()

,SUA_CODE_2016,SUA_NAME_2016,AREA_ALBERS_SQKM
0,1000,Not in any Significant Urban Area (NSW),787692.8137
1,1001,Albury - Wodonga,628.2471
2,1002,Armidale,274.5995
3,1003,Ballina,73.9758
4,1004,Batemans Bay,94.3022


In [100]:
df_con = pd.read_csv(filepath+"/NSW/concordance.csv")
df_con.head()

,GCCSA_CODE_2016,SA2_NAME_2016,SA3_NAME_2016,POA_CODE_2016,SSC_NAME_2016,SED_CODE_2016,DZN_CODE_2016,TZ16_CODE,SED_NAME_2016,LGA_NAME_2016,...,MB_CODE_2016,SA2_5DIGITCODE_2016,SA4_CODE_2016,SSC_CODE_2016,CED_NAME_2016,STATE_CODE_2016,SA1_MAINCODE_2016,TR_NAME_2016,SA4_NAME_2016,Dwellings 2016
0,1GSYD,Acacia Gardens,Blacktown - North,2763,Acacia Gardens,10071,115620001,3930,Riverstone,Blacktown (C),...,10092640000,11562,116,10014,Greenway,1,11602156201,Sydney,Sydney - Blacktown,44.0
1,1GSYD,Acacia Gardens,Blacktown - North,2763,Acacia Gardens,10071,115620001,3930,Riverstone,Blacktown (C),...,10092662000,11562,116,10014,Greenway,1,11602156201,Sydney,Sydney - Blacktown,60.0
2,1GSYD,Acacia Gardens,Blacktown - North,2763,Acacia Gardens,10071,115620001,3930,Riverstone,Blacktown (C),...,10092470000,11562,116,10014,Greenway,1,11602156202,Sydney,Sydney - Blacktown,44.0
3,1GSYD,Acacia Gardens,Blacktown - North,2763,Acacia Gardens,10071,115620001,3930,Riverstone,Blacktown (C),...,10092530000,11562,116,10014,Greenway,1,11602156202,Sydney,Sydney - Blacktown,35.0
4,1GSYD,Acacia Gardens,Blacktown - North,2763,Acacia Gardens,10071,115620001,3930,Riverstone,Blacktown (C),...,10092540000,11562,116,10014,Greenway,1,11602156202,Sydney,Sydney - Blacktown,37.0


In [101]:
df_sel = df_con[['SA2_NAME_2016', 'POA_CODE_2016','LGA_NAME_2016']].drop_duplicates().reset_index(drop=True)
df_sel.head()

,SA2_NAME_2016,POA_CODE_2016,LGA_NAME_2016
0,Acacia Gardens,2763,Blacktown (C)
1,Arncliffe - Bardwell Valley,2205,Rockdale (C)
2,Arncliffe - Bardwell Valley,2207,Rockdale (C)
3,Ashcroft - Busby - Miller,2168,Liverpool (C)
4,Ashfield,2130,Inner West (A)


In [102]:
to_check_duplicates(df_sel, 'POA_CODE_2016').sort_values(by=['POA_CODE_2016'])

,SA2_NAME_2016,POA_CODE_2016,LGA_NAME_2016
556,Potts Point - Woolloomooloo,2000,Sydney (C)
660,Sydney - Haymarket - The Rocks,2000,Sydney (C)
486,Newtown - Camperdown - Darlington,2008,Sydney (C)
575,Redfern - Chippendale,2008,Sydney (C)
654,Surry Hills,2010,Sydney (C)
...,...,...,...
1140,Hay,2878,Carrathool (A)
1028,Far West,2879,Central Darling (A)
1029,Far West,2879,Unincorporated NSW
863,Broken Hill,2880,Broken Hill (C)


In [103]:
POA_LGA = df_con[['POA_CODE_2016','LGA_NAME_2016']].drop_duplicates().reset_index(drop=True)
to_check_duplicates(POA_LGA, 'POA_CODE_2016').sort_values(by=['POA_CODE_2016'])

,POA_CODE_2016,LGA_NAME_2016
280,2018,Botany Bay (C)
362,2018,Sydney (C)
281,2020,Botany Bay (C)
352,2020,Inner West (A)
353,2020,Rockdale (C)
...,...,...
1007,2878,Balranald (A)
602,2879,Unincorporated NSW
601,2879,Central Darling (A)
478,2880,Broken Hill (C)


In [104]:
SA2_LGA = df_con[['SA2_NAME_2016','LGA_NAME_2016']].drop_duplicates().reset_index(drop=True)
to_check_duplicates(SA2_LGA, 'SA2_NAME_2016').sort_values(by=['SA2_NAME_2016'])

,SA2_NAME_2016,LGA_NAME_2016
376,Adamstown - Kotara,Newcastle (C)
377,Adamstown - Kotara,Lake Macquarie (C)
384,Albury Region,Albury (C)
385,Albury Region,Lockhart (A)
383,Albury Region,Greater Hume Shire (A)
...,...,...
374,Yarramundi - Londonderry,Hawkesbury (C)
773,Yass Region,Yass Valley (A)
774,Yass Region,Hilltops (A)
772,Yass Region,Upper Lachlan Shire (A)


There are many postalcodes with no unique LGA. There is one postalcode that has 10 different LGA associated.

In [105]:
POA_LGA['POA_CODE_2016'].value_counts().max() 

10

There are many postalcodes with no unique SA2.

In [106]:
to_check_duplicates(df_sel, 'SA2_NAME_2016') 

,SA2_NAME_2016,POA_CODE_2016,LGA_NAME_2016
1,Arncliffe - Bardwell Valley,2205,Rockdale (C)
2,Arncliffe - Bardwell Valley,2207,Rockdale (C)
4,Ashfield,2130,Inner West (A)
5,Ashfield,2131,Inner West (A)
6,Ashfield,2193,Inner West (A)
...,...,...,...
1759,Young Region,2666,Hilltops (A)
1760,Young Region,2794,Hilltops (A)
1761,Young Region,2803,Hilltops (A)
1762,Young Region,2807,Hilltops (A)


In [107]:
df_sel['POA_CODE_2016'] = df_sel['POA_CODE_2016'].astype(np.str)

In [108]:
merged.head()

,notification_date,postcode,lga_name19,total_population
0,2020-01-25,2134,Burwood (A),16593.0
1,2020-03-29,2134,Burwood (A),16593.0
2,2020-03-30,2134,Burwood (A),16593.0
3,2020-04-02,2134,Burwood (A),16593.0
4,2020-04-08,2134,Burwood (A),16593.0


In [109]:
df_sel.head()

,SA2_NAME_2016,POA_CODE_2016,LGA_NAME_2016
0,Acacia Gardens,2763,Blacktown (C)
1,Arncliffe - Bardwell Valley,2205,Rockdale (C)
2,Arncliffe - Bardwell Valley,2207,Rockdale (C)
3,Ashcroft - Busby - Miller,2168,Liverpool (C)
4,Ashfield,2130,Inner West (A)


In [110]:
SA2_SUA = pd.read_csv(filepath+"/NSW/SA2_SUA_2016_AUST.csv")
SA2_SUA = SA2_SUA[['SA2_NAME_2016', 'SUA_NAME_2016','AREA_ALBERS_SQKM']].drop_duplicates().reset_index(drop=True)
SA2_SUA.head()

,SA2_NAME_2016,SUA_NAME_2016,AREA_ALBERS_SQKM
0,Braidwood,Not in any Significant Urban Area (NSW),3418.3525
1,Karabar,Canberra - Queanbeyan,6.9825
2,Queanbeyan,Canberra - Queanbeyan,4.7634
3,Queanbeyan - East,Canberra - Queanbeyan,13.0034
4,Queanbeyan Region,Not in any Significant Urban Area (NSW),3054.4099


In [111]:
SA2_PAO_SUA= pd.merge(df_sel, SA2_SUA,on=['SA2_NAME_2016'], how='left').drop_duplicates().reset_index(drop=True)

In [112]:
SA2_PAO_SUA =SA2_PAO_SUA[['SA2_NAME_2016','POA_CODE_2016','LGA_NAME_2016','SUA_NAME_2016']]


SA2_PAO_SUA['SUA_NAME_2016'].replace(r"Not in any", np.nan, regex=True, inplace = True)
SA2_PAO_SUA.head()

,SA2_NAME_2016,POA_CODE_2016,LGA_NAME_2016,SUA_NAME_2016
0,Acacia Gardens,2763,Blacktown (C),Sydney
1,Arncliffe - Bardwell Valley,2205,Rockdale (C),Sydney
2,Arncliffe - Bardwell Valley,2207,Rockdale (C),Sydney
3,Ashcroft - Busby - Miller,2168,Liverpool (C),Sydney
4,Ashfield,2130,Inner West (A),Sydney


In [113]:
len(merged.axes[0])
merged = merged.reset_index(drop=True)

In [114]:
merged.iloc[20:30, :]

,notification_date,postcode,lga_name19,total_population
20,2021-08-02,2134,Burwood (A),16593.0
21,2021-08-04,2134,Burwood (A),16593.0
22,2021-08-04,2134,Burwood (A),16593.0
23,2021-08-06,2134,Burwood (A),16593.0
24,2021-08-06,2134,Burwood (A),16593.0
25,2021-08-06,2134,Burwood (A),16593.0
26,2021-08-09,2134,Burwood (A),16593.0
27,2021-08-11,2134,Burwood (A),16593.0
28,2021-08-12,2134,Burwood (A),16593.0
29,2021-08-14,2134,Burwood (A),16593.0


In [115]:
test = merged.copy()
test

,notification_date,postcode,lga_name19,total_population
0,2020-01-25,2134,Burwood (A),16593.0
1,2020-03-29,2134,Burwood (A),16593.0
2,2020-03-30,2134,Burwood (A),16593.0
3,2020-04-02,2134,Burwood (A),16593.0
4,2020-04-08,2134,Burwood (A),16593.0
...,...,...,...,...
72438,2021-10-20,2645,Federation (A),399.0
72439,2021-10-21,2834,Walgett (A),2695.0
72440,2021-10-23,2665,Coolamon (A),2179.0
72441,2021-10-23,2665,Coolamon (A),2179.0


In [116]:
df_cut = test.groupby('postcode')['notification_date'].count().reset_index()
merged_t = pd.merge(df_cut, test, on=['postcode'], how='outer').drop_duplicates().reset_index(drop=True)

merged_t = merged_t.rename(columns={'notification_date_x':'cases','total_population':'population'})
merged_t = merged_t.drop(columns=['notification_date_y']).drop_duplicates().reset_index(drop=True)
merged_t

,postcode,cases,lga_name19,population
0,2000,241,Sydney (C),52601.0
1,2000,241,Correctional settings,52601.0
2,2007,46,Sydney (C),10023.0
3,2008,59,Sydney (C),13373.0
4,2009,85,Sydney (C),14748.0
...,...,...,...,...
556,2869,4,Parkes (A),1451.0
557,2870,11,Parkes (A),11918.0
558,2871,4,Forbes (A),9416.0
559,2880,130,Broken Hill (C),18659.0


Delete rows 'Correctional settings' as lga.

In [117]:
NSW = merged_t[merged_t.lga_name19 != 'Correctional settings']
NSW

,postcode,cases,lga_name19,population
0,2000,241,Sydney (C),52601.0
2,2007,46,Sydney (C),10023.0
3,2008,59,Sydney (C),13373.0
4,2009,85,Sydney (C),14748.0
5,2010,294,Sydney (C),30102.0
...,...,...,...,...
555,2869,4,Narromine (A),1451.0
556,2869,4,Parkes (A),1451.0
557,2870,11,Parkes (A),11918.0
558,2871,4,Forbes (A),9416.0


In [118]:
#for POA in NSW, 
NSW['SA2_NAME_2016'] = np.nan
NSW['SUA_NAME_2016'] = np.nan
i = 0
for row in range(len(NSW.axes[0])):
    pstc = NSW['postcode'].iloc[i]
    lga = NSW['lga_name19'].iloc[i]

    tmp = SA2_PAO_SUA[(SA2_PAO_SUA['POA_CODE_2016'] == pstc ) & (SA2_PAO_SUA['LGA_NAME_2016'] == lga)]
    array_SA2 = tmp['SA2_NAME_2016'].unique()
    array_SUA = tmp['SUA_NAME_2016'].unique()
    array_SA2  = [x for x in array_SA2  if str(x) != 'nan']
    array_SUA = [x for x in array_SUA if str(x) != 'nan']
    
    NSW['SA2_NAME_2016'][i] = array_SA2
    NSW['SUA_NAME_2016'][i] = array_SUA
    i += 1

/var/folders/2r/dxh5vfq12h7bhlg2f88kpl440000gn/T/ipykernel_24272/2822599217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NSW['SA2_NAME_2016'] = np.nan
/var/folders/2r/dxh5vfq12h7bhlg2f88kpl440000gn/T/ipykernel_24272/2822599217.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NSW['SUA_NAME_2016'] = np.nan
/var/folders/2r/dxh5vfq12h7bhlg2f88kpl440000gn/T/ipykernel_24272/2822599217.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in th

In [119]:
NSW.head()

,postcode,cases,lga_name19,population,SA2_NAME_2016,SUA_NAME_2016
0,2000,241,Sydney (C),52601.0,"[Potts Point - Woolloomooloo, Sydney - Haymark...",[Sydney]
2,2007,46,Sydney (C),10023.0,"[Newtown - Camperdown - Darlington, Redfern - ...",[Sydney]
3,2008,59,Sydney (C),13373.0,[Pyrmont - Ultimo],[Sydney]
4,2009,85,Sydney (C),14748.0,"[Darlinghurst, Surry Hills]",[Sydney]
5,2010,294,Sydney (C),30102.0,"[Potts Point - Woolloomooloo, Sydney - Haymark...",[Sydney]


In [121]:
NSW.to_pickle(saving_path+"/AllScales24Oct.pkl")

In [122]:
AllScales = pd.read_pickle(saving_path+"AllScales24Oct.pkl")

In [123]:
AllScales.shape

(502, 6)

In [124]:
AllScales['cases'].sum()

73162

Multiple SUA codes per postcode:

In [125]:
multiSUA = AllScales[AllScales['SUA_NAME_2016'].str.len() > 1]
multiSUA

,postcode,cases,lga_name19,population,SA2_NAME_2016,SUA_NAME_2016
200,2217,256,Bayside (A),27819.0,"[Lake Munmorah - Mannering Park, Morisset - Co...","[Central Coast, Morisset - Cooranbong, Newcast..."
251,2291,53,Newcastle (C),13005.0,"[Nelson Bay Peninsula, Raymond Terrace, Seaham...","[Nelson Bay, Newcastle - Maitland]"
302,2350,11,Armidale Regional (A),25243.0,"[Grafton, Grafton Region, Woolgoolga - Arrawarra]","[Grafton, Coffs Harbour]"
347,2470,15,Richmond Valley (A),15154.0,"[Berry - Kangaroo Valley, Callala Bay - Currar...","[St Georges Basin - Sanctuary Point, Nowra - B..."


There are 4 postcodes with multiple SUA.

In [126]:
lost_ptc = multiSUA['postcode'].value_counts().to_frame()
lost_ptc.count()

postcode    4
dtype: int64

19,765 cases without SUA associated to the postcode.

In [127]:
mask = (AllScales['SUA_NAME_2016'].str.len() == 0) 
AllScales[mask]['cases'].sum()

19765

## POA and SUA

In [129]:
AllScales = pd.read_pickle(saving_path+"/AllScales24Oct.pkl")
AllScales = AllScales.reset_index(drop=True)
AllScales.head()

,postcode,cases,lga_name19,population,SA2_NAME_2016,SUA_NAME_2016
0,2000,241,Sydney (C),52601.0,"[Potts Point - Woolloomooloo, Sydney - Haymark...",[Sydney]
1,2007,46,Sydney (C),10023.0,"[Newtown - Camperdown - Darlington, Redfern - ...",[Sydney]
2,2008,59,Sydney (C),13373.0,[Pyrmont - Ultimo],[Sydney]
3,2009,85,Sydney (C),14748.0,"[Darlinghurst, Surry Hills]",[Sydney]
4,2010,294,Sydney (C),30102.0,"[Potts Point - Woolloomooloo, Sydney - Haymark...",[Sydney]


In [130]:
AllScales.shape
AllScales['cases'].sum()

73162

In [131]:
df_s = AllScales.copy()
print(df_s.shape)
df_s['SUA_SA2_LGA_NAME'] = np.nan

(502, 6)


In [132]:
df_s[df_s['SUA_NAME_2016'].isna()].shape

(53, 7)

In [133]:
i = 0
r = range(len(df_s.axes[0]))
'''It will delete the "Not any" in SUA. If there is one SUA, it will assign it at the 
General name ['SUA_SA2_LGA_NAME']'''
for row in r:
    #Obtain a list of the separated elements. If it contains 'Not in any' it is replaced with nan.
    SUA_name = df_s['SUA_NAME_2016'].iloc[i]
    try:
    #if np.size(SUA_name) > 1) | (np.size(SUA_name) == 1):
        SUA_name = list(SUA_name)
        #not_any = 'Not in any Significant Urban Area (NSW)'
        #if not_any in SUA_name:
        #    SUA_name.remove(not_any)
        if len(SUA_name) == 1:
            df_s['SUA_SA2_LGA_NAME'][i] = SUA_name[0]
        elif len(SUA_name) == 0 | (df_s['SUA_SA2_LGA_NAME'][i] == 'NaN'):
            SA2_name = df_s['SA2_NAME_2016'].iloc[i]
            SA2_name = list(SA2_name)
            if len(SA2_name) == 1:
                df_s['SUA_SA2_LGA_NAME'].iloc[i] = SA2_name[0]  + ' (SA2)'
            elif len(SA2_name) == 0:
                LGA_name = df_s['lga_name19'].iloc[i]
                df_s['SUA_SA2_LGA_NAME'].iloc[i] = LGA_name  + ' (LGA)'
            else:
                #pick one at random
                SA2_name = df_s['SA2_NAME_2016'].iloc[i]
                SA2_name = list(SA2_name)
                random_name = random.choice(SA2_name)
                df_s['SUA_SA2_LGA_NAME'].iloc[i] = random_name  + ' (SA2)'
        #len(SUA_name) > 1) |  multiple SUA?
        i += 1
    except TypeError:
    #else:
        LGA_name = df_s['lga_name19'].iloc[i]
        df_s['SUA_SA2_LGA_NAME'].iloc[i] = LGA_name  + ' (LGA)'

        i += 1

/var/folders/2r/dxh5vfq12h7bhlg2f88kpl440000gn/T/ipykernel_24272/4239607520.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['SUA_SA2_LGA_NAME'][i] = SUA_name[0]
/Users/jimenaspi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [134]:
df_s.shape

(502, 7)

In [135]:
df_s.loc[pd.isnull(df_s.SUA_SA2_LGA_NAME)]['cases'].sum() 

335

In [136]:
df_s.iloc[51:100,2:]

,lga_name19,population,SA2_NAME_2016,SUA_NAME_2016,SUA_SA2_LGA_NAME
51,Ku-ring-gai (A),14191.0,"[Wahroonga (East) - Warrawee, Waitara - Wahroo...",[Sydney],Sydney
52,Ku-ring-gai (A),13473.0,"[Asquith - Mount Colah, Hornsby - East, Hornsb...",[Sydney],Sydney
53,Ku-ring-gai (A),13701.0,[Asquith - Mount Colah],[Sydney],Sydney
54,Ku-ring-gai (A),7732.0,"[Asquith - Mount Colah, Berowra - Brooklyn - C...",[Sydney],Sydney
55,Ku-ring-gai (A),16447.0,[Berowra - Brooklyn - Cowan],[Sydney],Sydney
56,Ku-ring-gai (A),22166.0,"[Berowra - Brooklyn - Cowan, Galston - Laughto...",[Sydney],Sydney
57,Ku-ring-gai (A),20535.0,[Berowra - Brooklyn - Cowan],[Sydney],Sydney
58,Ku-ring-gai (A),25215.0,"[Bayview - Elanora Heights, Terrey Hills - Duf...",[Sydney],Sydney
59,Hornsby (A),38470.0,[Frenchs Forest - Belrose],[Sydney],Sydney
60,Hornsby (A),7007.0,"[Frenchs Forest - Belrose, Manly Vale - Allamb...",[Sydney],Sydney


In [137]:
check_nan = df_s[df_s['SUA_SA2_LGA_NAME'].isna()]
print(check_nan.shape)
check_nan

(4, 7)


,postcode,cases,lga_name19,population,SA2_NAME_2016,SUA_NAME_2016,SUA_SA2_LGA_NAME
176,2217,256,Bayside (A),27819.0,"[Lake Munmorah - Mannering Park, Morisset - Co...","[Central Coast, Morisset - Cooranbong, Newcast...",NaN
220,2291,53,Newcastle (C),13005.0,"[Nelson Bay Peninsula, Raymond Terrace, Seaham...","[Nelson Bay, Newcastle - Maitland]",NaN
267,2350,11,Armidale Regional (A),25243.0,"[Grafton, Grafton Region, Woolgoolga - Arrawarra]","[Grafton, Coffs Harbour]",NaN
308,2470,15,Richmond Valley (A),15154.0,"[Berry - Kangaroo Valley, Callala Bay - Currar...","[St Georges Basin - Sanctuary Point, Nowra - B...",NaN


In [138]:
check_nan.cases.sum()

335

In [139]:
df_s[df_s['cases'] == 3 ]

,postcode,cases,lga_name19,population,SA2_NAME_2016,SUA_NAME_2016,SUA_SA2_LGA_NAME
276,2381,3,Gunnedah (A),1174.0,[Tenterfield],[],Tenterfield (SA2)
277,2400,3,Moree Plains (A),10593.0,[Ballina Region],[],Ballina Region (SA2)
278,2420,3,Dungog (A),3939.0,"[Ballina, Ballina Region, Lennox Head - Skenna...",[Ballina],Ballina
296,2448,3,Nambucca (A),9254.0,[Balgownie - Fairy Meadow],[Wollongong],Wollongong
303,2462,3,Clarence Valley (A),2523.0,[Kiama Hinterland - Gerringong],[],Kiama Hinterland - Gerringong (SA2)
304,2463,3,Clarence Valley (A),7718.0,"[Berry - Kangaroo Valley, Kiama Hinterland - G...",[],Berry - Kangaroo Valley (SA2)
319,2484,3,Tweed (A),19094.0,[Rosemeadow - Glen Alpine],[Sydney],Sydney
320,2485,3,Tweed (A),15464.0,[Macquarie Fields - Glenfield],[Sydney],Sydney
388,2626,3,Snowy Monaro Regional (A),482.0,[Kurrajong Heights - Ebenezer],[Sydney],Sydney
396,2643,3,Federation (A),2697.0,"[Doonside - Woodcroft, Rooty Hill - Minchinbury]",[Sydney],Sydney


In [140]:
df_s_n = df_s.copy()
df_s_n.shape 

(502, 7)

73,162 cases remain

In [141]:
df_s_n.cases.sum()

73162

In [142]:
df_s_n[df_s_n['SUA_SA2_LGA_NAME'].isna()].count()

postcode            4
cases               4
lga_name19          4
population          4
SA2_NAME_2016       4
SUA_NAME_2016       4
SUA_SA2_LGA_NAME    0
dtype: int64

In [143]:
df_s_n = df_s_n.drop(columns = ['lga_name19','SA2_NAME_2016','SUA_NAME_2016'])
df_s_n = df_s_n.drop_duplicates().reset_index(drop=True)

Keep only one row, to not have duplicated information.

In [144]:
to_check_duplicates(df_s_n, 'postcode')

,postcode,cases,population,SUA_SA2_LGA_NAME
245,2323,104,25428.0,Bulahdelah - Stroud (SA2)
246,2323,104,25428.0,Taree Region (SA2)
247,2324,97,23431.0,Forster - Tuncurry
248,2324,97,23431.0,Taree
443,2775,3,1491.0,Forbes (SA2)
444,2775,3,1491.0,Broken Hill
473,2831,20,1945.0,Dubbo Regional (A) (LGA)
474,2831,20,1945.0,Warren (A) (LGA)
493,2869,4,1451.0,Narromine (A) (LGA)
494,2869,4,1451.0,Parkes (A) (LGA)


In [145]:
without_dupl = df_s_n.drop([237 , 240, 396, 426, 446])
to_check_duplicates(without_dupl, 'postcode')

,postcode,cases,population,SUA_SA2_LGA_NAME
245,2323,104,25428.0,Bulahdelah - Stroud (SA2)
246,2323,104,25428.0,Taree Region (SA2)
247,2324,97,23431.0,Forster - Tuncurry
248,2324,97,23431.0,Taree
443,2775,3,1491.0,Forbes (SA2)
444,2775,3,1491.0,Broken Hill
473,2831,20,1945.0,Dubbo Regional (A) (LGA)
474,2831,20,1945.0,Warren (A) (LGA)
493,2869,4,1451.0,Narromine (A) (LGA)
494,2869,4,1451.0,Parkes (A) (LGA)


In [146]:
without_dupl= without_dupl.drop(columns = ['postcode'])

In [148]:
without_dupl = without_dupl.reset_index(drop=True)
without_dupl.to_pickle(saving_path+"/NSW_SUA_SA2_LGA24Oct.pkl")

# Chile dataset

# To work with dates: 
Incremental dataset includes cases by date.

In [150]:
chile = pd.read_csv(filepath+'Chile/Covid-19ChileIncremental092321.csv')

In [151]:
chile.head()

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,2020-04-08,...,2021-08-20,2021-08-23,2021-08-27,2021-08-30,2021-09-03,2021-09-06,2021-09-10,2021-09-13,2021-09-17,Tasa
0,Arica y Parinacota,15,Arica,15101.0,247552.0,6.0,6.0,12.0,41.0,63.0,...,29501.0,29535.0,29557.0,29579.0,29662.0,29681.0,29763.0,29836.0,29929.0,12090.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,0.0,0.0,0.0,0.0,0.0,...,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,70.0,5677.2
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,0.0,0.0,0.0,0.0,0.0,...,87.0,87.0,87.0,87.0,88.0,88.0,88.0,88.0,88.0,10864.2
3,Arica y Parinacota,15,Putre,15201.0,2515.0,0.0,0.0,0.0,0.0,0.0,...,202.0,202.0,203.0,204.0,204.0,204.0,205.0,207.0,208.0,8270.4
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,229.0,230.0,230.0,232.0,155.0,156.0,158.0,163.0,161.0,NaN


In [152]:
chile = chile.drop(columns=['Region','Codigo region','Codigo comuna', 'Tasa'])
chile.head()

,Comuna,Poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,2020-04-08,2020-04-10,2020-04-13,2020-04-15,...,2021-08-16,2021-08-20,2021-08-23,2021-08-27,2021-08-30,2021-09-03,2021-09-06,2021-09-10,2021-09-13,2021-09-17
0,Arica,247552.0,6.0,6.0,12.0,41.0,63.0,87.0,115.0,124.0,...,29493.0,29501.0,29535.0,29557.0,29579.0,29662.0,29681.0,29763.0,29836.0,29929.0
1,Camarones,1233.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,69.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,70.0
2,General Lagos,810.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,87.0,87.0,87.0,87.0,87.0,88.0,88.0,88.0,88.0,88.0
3,Putre,2515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,201.0,202.0,202.0,203.0,204.0,204.0,204.0,205.0,207.0,208.0
4,Desconocido Arica y Parinacota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,229.0,229.0,230.0,230.0,232.0,155.0,156.0,158.0,163.0,161.0


In [153]:
chile_m = pd.melt(chile, id_vars=chile.iloc[:,0:2], value_vars = chile.iloc[:,3:-1],var_name='date',value_name='cases',ignore_index=True)

In [154]:
chile_m['date']= pd.to_datetime(chile_m['date'])
chile_m = chile_m.sort_values(by='date').reset_index(drop=True)
chile_m.head() #Cases are incremental!! not daily

,Comuna,Poblacion,date,cases
0,Arica,247552.0,2020-04-01,6.0
1,Mulchen,31041.0,2020-04-01,5.0
2,Lota,45750.0,2020-04-01,0.0
3,Los Angeles,218515.0,2020-04-01,15.0
4,Los Alamos,22524.0,2020-04-01,0.0


In [155]:
type(chile_m['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [156]:
chile_m.cases.sum()

134636036.0

In [157]:
chile_m['Comuna'].unique() #Delete Desconocido

array(['Arica', 'Mulchen', 'Lota', 'Los Angeles', 'Los Alamos', 'Lebu',
       'Laja', 'Hualqui', 'Hualpen', 'Nacimiento', 'Florida', 'Coronel',
       'Contulmo', 'Concepcion', 'Chiguayante', 'Canete', 'Cabrero',
       'Arauco', 'Antuco', 'Curanilahue', 'Negrete', 'Penco', 'Quilaco',
       'Ercilla', 'Curarrehue', 'Curacautin', 'Cunco', 'Collipulli',
       'Cholchol', 'Carahue', 'Angol', 'Desconocido Biobio', 'Yumbel',
       'Tucapel', 'Tome', 'Tirua', 'Talcahuano', 'Santa Juana',
       'Santa Barbara', 'San Rosendo', 'San Pedro de la Paz', 'Quilleco',
       'Alto Biobio', 'Desconocido Nuble', 'Yungay', 'Treguaco', 'Teno',
       'Talca', 'San Rafael', 'San Javier', 'San Clemente',
       'Sagrada Familia', 'Romeral', 'Rio Claro', 'Retiro', 'Rauco',
       'Pencahue', 'Pelluhue', 'Pelarco', 'Parral', 'Molina', 'Maule',
       'Longavi', 'Linares', 'Licanten', 'Vichuquen', 'Freire',
       'Villa Alegre', 'Desconocido Maule', 'San Nicolas', 'San Ignacio',
       'San Fabian', 'Sa

In [158]:
unknown_com = chile_m['Comuna'].str.contains('Desconocid', regex=True)

In [159]:
chile_m[~unknown_com]['cases'].sum() 

132426176.0

In [160]:
chile_m = chile_m[~unknown_com] #Incremental. Should be around 1.6 million up to date.

In [161]:
chile_m.head()

,Comuna,Poblacion,date,cases
0,Arica,247552.0,2020-04-01,6.0
1,Mulchen,31041.0,2020-04-01,5.0
2,Lota,45750.0,2020-04-01,0.0
3,Los Angeles,218515.0,2020-04-01,15.0
4,Los Alamos,22524.0,2020-04-01,0.0


In [162]:
chile_m = chile_m[chile_m['cases'] != 0]
chile_m['cases'].min() #To check consistency

1.0

In [163]:
chile_m['Poblacion'].min() #To check consistency

137.0

In [164]:
chile_m[chile_m['Poblacion'].isna()] #To check nan values

,Comuna,Poblacion,date,cases


In [165]:
chile_m[chile_m['Comuna']=='Algarrobo']

,Comuna,Poblacion,date,cases
198,Algarrobo,15174.0,2020-04-01,7.0
559,Algarrobo,15174.0,2020-04-03,9.0
921,Algarrobo,15174.0,2020-04-06,9.0
1283,Algarrobo,15174.0,2020-04-08,8.0
1645,Algarrobo,15174.0,2020-04-10,8.0
...,...,...,...,...
54135,Algarrobo,15174.0,2021-08-30,1166.0
54498,Algarrobo,15174.0,2021-09-03,1167.0
54859,Algarrobo,15174.0,2021-09-06,1169.0
55220,Algarrobo,15174.0,2021-09-10,1173.0


To get daily cases

In [166]:
chile_m.sort_values(by = ['Comuna', 'date'], inplace = True)
chile_m['diff_cases'] = chile_m['cases'].diff()
mask = chile_m.Comuna != chile_m.Comuna.shift(1)
chile_m['diff_cases'][mask] = chile_m['cases']

/var/folders/2r/dxh5vfq12h7bhlg2f88kpl440000gn/T/ipykernel_24272/1891254276.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chile_m['diff_cases'][mask] = chile_m['cases']


In [167]:
m = chile_m[chile_m['Comuna']=='Algarrobo']

In [168]:
m

,Comuna,Poblacion,date,cases,diff_cases
198,Algarrobo,15174.0,2020-04-01,7.0,7.0
559,Algarrobo,15174.0,2020-04-03,9.0,2.0
921,Algarrobo,15174.0,2020-04-06,9.0,0.0
1283,Algarrobo,15174.0,2020-04-08,8.0,-1.0
1645,Algarrobo,15174.0,2020-04-10,8.0,0.0
...,...,...,...,...,...
54135,Algarrobo,15174.0,2021-08-30,1166.0,4.0
54498,Algarrobo,15174.0,2021-09-03,1167.0,1.0
54859,Algarrobo,15174.0,2021-09-06,1169.0,2.0
55220,Algarrobo,15174.0,2021-09-10,1173.0,4.0


In [169]:
m.diff_cases.sum()

1174.0

In [170]:
chile_m.diff_cases.sum() 

1958409.0

In [171]:
chile_m[chile_m['diff_cases'] < 0].sum() #There is an adjustment of -2335

/var/folders/2r/dxh5vfq12h7bhlg2f88kpl440000gn/T/ipykernel_24272/1572737144.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  chile_m[chile_m['diff_cases'] < 0].sum() #There is an adjustment of -2335


Comuna        AlgarroboAlgarroboAlhueAlhueAlhueAlhueAlhueAlt...
Poblacion                                            15122674.0
cases                                                  385162.0
diff_cases                                              -2335.0
dtype: object

In [172]:
negative_cases = chile_m[chile_m['diff_cases'] < 0].sort_values(by=['diff_cases'])
negative_cases.head() #Top negative cases

,Comuna,Poblacion,date,cases,diff_cases
12153,Valparaiso,315732.0,2020-07-20,3310.0,-196.0
12151,Vina del Mar,361371.0,2020-07-20,3179.0,-158.0
12161,Quilpue,167085.0,2020-07-20,1278.0,-130.0
12162,Quillota,97572.0,2020-07-20,1209.0,-126.0
12152,Villa Alemana,139310.0,2020-07-20,1206.0,-102.0


The adjustment of negative cases will be dropped.

In [173]:
negative_mask = chile_m['diff_cases'] < 0
chile_m = chile_m[~negative_mask] 

In [174]:
chile_m

,Comuna,Poblacion,date,cases,diff_cases
198,Algarrobo,15174.0,2020-04-01,7.0,7.0
559,Algarrobo,15174.0,2020-04-03,9.0,2.0
921,Algarrobo,15174.0,2020-04-06,9.0,0.0
1645,Algarrobo,15174.0,2020-04-10,8.0,0.0
2008,Algarrobo,15174.0,2020-04-13,8.0,0.0
...,...,...,...,...,...
54143,Zapallar,7994.0,2021-08-30,593.0,0.0
54506,Zapallar,7994.0,2021-09-03,594.0,1.0
54867,Zapallar,7994.0,2021-09-06,594.0,0.0
55228,Zapallar,7994.0,2021-09-10,594.0,0.0


In [176]:
chile_m.to_csv(saving_path+'/Chile_wdates23Sept.csv', index = False)

# Chile dataset without dates.

In [177]:
chile = pd.read_csv(filepath+'/Chile/2021-10-22-CasosConfirmados.csv')

In [178]:
chile

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,Casos Confirmados
0,Arica y Parinacota,15,Arica,15101.0,247552.0,30828.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,71.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,88.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,215.0
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,204.0
...,...,...,...,...,...,...
357,Magallanes,12,Rio Verde,12103.0,211.0,17.0
358,Magallanes,12,San Gregorio,12104.0,681.0,74.0
359,Magallanes,12,Timaukel,12303.0,282.0,42.0
360,Magallanes,12,Torres del Paine,12402.0,1021.0,22.0


In [179]:
unknown_com = chile['Comuna'].str.contains('Desconocid', regex=True)

In [180]:
chile[unknown_com]['Casos Confirmados'].sum()

18458.0

Total cases: 2,018,241.0

In [181]:
chile['Casos Confirmados'].sum() 

2018241.0

In [182]:
round(chile[unknown_com]['Casos Confirmados'].sum()*100/chile['Casos Confirmados'].sum() , 4)

0.9146

0.9146% of the data will be dropped because the 'Comuna' is unknown.

In [183]:
chile = chile[~unknown_com]

In [184]:
chile

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,Casos Confirmados
0,Arica y Parinacota,15,Arica,15101.0,247552.0,30828.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,71.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,88.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,215.0
5,Tarapacá,1,Alto Hospicio,1107.0,129999.0,16115.0
...,...,...,...,...,...,...
356,Magallanes,12,Punta Arenas,12101.0,141984.0,25188.0
357,Magallanes,12,Rio Verde,12103.0,211.0,17.0
358,Magallanes,12,San Gregorio,12104.0,681.0,74.0
359,Magallanes,12,Timaukel,12303.0,282.0,42.0


In [185]:
chile = chile.drop(columns=['Region', 'Codigo region', 'Codigo comuna'])
chile = chile.rename({'Poblacion':'population','Casos Confirmados':'cases'}, axis='columns')

In [186]:
chile

,Comuna,population,cases
0,Arica,247552.0,30828.0
1,Camarones,1233.0,71.0
2,General Lagos,810.0,88.0
3,Putre,2515.0,215.0
5,Alto Hospicio,129999.0,16115.0
...,...,...,...
356,Punta Arenas,141984.0,25188.0
357,Rio Verde,211.0,17.0
358,San Gregorio,681.0,74.0
359,Timaukel,282.0,42.0


In [187]:
chile.to_csv(saving_path+'/Chile_26Oct.csv', index = False)

# Brazil

In [7]:
brazil = pd.read_csv(filepath+'/Brazil/cases-brazil-cities.csv')

To select the information that will be usefil in the analysis: City, Cases and population.

In [8]:
brazil = brazil.loc[:,['ibgeID','city', 'totalCases']]
brazil

,ibgeID,city,totalCases
0,5200050,Abadia de Goiás/GO,1919
1,3100104,Abadia dos Dourados/MG,582
2,5200100,Abadiânia/GO,546
3,1500107,Abaetetuba/PA,9444
4,3100203,Abaeté/MG,1665
...,...,...,...
5591,4301552,Áurea/RS,331
5592,4101150,Ângulo/PR,381
5593,2900504,Érico Cardoso/BA,329
5594,1505106,Óbidos/PA,6654


In [9]:
to_check_duplicates(brazil, 'ibgeID') #no duplicate cities

,ibgeID,city,totalCases


In [10]:
brazil['ibgeID'].isnull().values.any() #No null values

False

Brazil population information:

In [11]:
headers = ['city', 'ibgeID', 'population']
dtypes = {'name_municipio':str, 'ibgeID':str,'population':float}

brazil_pop = pd.read_csv(filepath+'/Brazil/estimativa_dou_2021edited.csv',skiprows = [0], names=headers, dtype=dtypes)

In [12]:
brazil_pop 

,city,ibgeID,population
0,Rodelas/BA,2927101,9548.0
1,Ibiassucê/BA,2912004,8849.0
2,Tabatinga/AM,1304062,68502.0
3,Jacareacanga/PA,1503754,6952.0
4,Porto Velho/RO,1100205,548952.0
...,...,...,...
5565,União da Serra/RS,4322350,1084.0
5566,Engenho Velho/RS,4306924,932.0
5567,Araguainha/MT,5101209,909.0
5568,Borá/SP,3507209,839.0


In [13]:
brazil_pop[brazil_pop['ibgeID'] == '5200050']

,city,ibgeID,population
3242,Abadia de Goiás/GO,5200050,9158.0


# Merge the population and the cases information. 
Fill the places without cases with 0.

In [14]:
brazil['ibgeID']= brazil['ibgeID'].astype(np.str)
brazil_m = brazil.merge(brazil_pop, on = 'ibgeID' , how='outer', copy = False)

In [15]:
brazil_m

,ibgeID,city_x,totalCases,city_y,population
0,5200050,Abadia de Goiás/GO,1919,Abadia de Goiás/GO,9158.0
1,3100104,Abadia dos Dourados/MG,582,Abadia dos Dourados/MG,7022.0
2,5200100,Abadiânia/GO,546,Abadiânia/GO,20873.0
3,1500107,Abaetetuba/PA,9444,Abaetetuba/PA,160439.0
4,3100203,Abaeté/MG,1665,Abaeté/MG,23263.0
...,...,...,...,...,...
5591,4301552,Áurea/RS,331,Áurea/RS,3517.0
5592,4101150,Ângulo/PR,381,Ângulo/PR,2931.0
5593,2900504,Érico Cardoso/BA,329,Érico Cardoso/BA,10513.0
5594,1505106,Óbidos/PA,6654,Óbidos/PA,52473.0


In [16]:
brazil_m[brazil_m['totalCases']==0]

,ibgeID,city_x,totalCases,city_y,population
781,12,CASO SEM LOCALIZAÇÃO DEFINIDA/AC,0,NaN,NaN
782,27,CASO SEM LOCALIZAÇÃO DEFINIDA/AL,0,NaN,NaN
783,13,CASO SEM LOCALIZAÇÃO DEFINIDA/AM,0,NaN,NaN
784,16,CASO SEM LOCALIZAÇÃO DEFINIDA/AP,0,NaN,NaN
789,21,CASO SEM LOCALIZAÇÃO DEFINIDA/MA,0,NaN,NaN
791,50,CASO SEM LOCALIZAÇÃO DEFINIDA/MS,0,NaN,NaN
794,25,CASO SEM LOCALIZAÇÃO DEFINIDA/PB,0,NaN,NaN
795,26,CASO SEM LOCALIZAÇÃO DEFINIDA/PE,0,NaN,NaN
798,33,CASO SEM LOCALIZAÇÃO DEFINIDA/RJ,0,NaN,NaN
799,24,CASO SEM LOCALIZAÇÃO DEFINIDA/RN,0,NaN,NaN


No places with 0 cases. Cases without localization will be deleted from the dataset.

In [17]:
to_check_duplicates(brazil_m, 'ibgeID')

,ibgeID,city_x,totalCases,city_y,population


To save the file with cities, cases and population:

In [18]:
brazil_m

,ibgeID,city_x,totalCases,city_y,population
0,5200050,Abadia de Goiás/GO,1919,Abadia de Goiás/GO,9158.0
1,3100104,Abadia dos Dourados/MG,582,Abadia dos Dourados/MG,7022.0
2,5200100,Abadiânia/GO,546,Abadiânia/GO,20873.0
3,1500107,Abaetetuba/PA,9444,Abaetetuba/PA,160439.0
4,3100203,Abaeté/MG,1665,Abaeté/MG,23263.0
...,...,...,...,...,...
5591,4301552,Áurea/RS,331,Áurea/RS,3517.0
5592,4101150,Ângulo/PR,381,Ângulo/PR,2931.0
5593,2900504,Érico Cardoso/BA,329,Érico Cardoso/BA,10513.0
5594,1505106,Óbidos/PA,6654,Óbidos/PA,52473.0


In [19]:
brazil_m = brazil_m[['city_x', 'population','totalCases']]
brazil_m

,city_x,population,totalCases
0,Abadia de Goiás/GO,9158.0,1919
1,Abadia dos Dourados/MG,7022.0,582
2,Abadiânia/GO,20873.0,546
3,Abaetetuba/PA,160439.0,9444
4,Abaeté/MG,23263.0,1665
...,...,...,...
5591,Áurea/RS,3517.0,331
5592,Ângulo/PR,2931.0,381
5593,Érico Cardoso/BA,10513.0,329
5594,Óbidos/PA,52473.0,6654


In [20]:
brazil_m.totalCases.sum()

21594660

In [202]:
brazil_m.to_csv(saving_path+'/Brazil24Oct.csv' , header=['city', 'population', 'cases'] ,index = False)

# USA

In [203]:
dtypes = {'date': str, 'county':str, 'state':str, 'fips':float, 'cases':float, 'deaths':float}
usa = pd.read_csv(filepath+'/USA/us-counties.csv', dtype = dtypes,
                  parse_dates= ['date'])

In [204]:
print(usa['date'].min())
print(usa['date'].max()) #Last date: October 13th 2021

2020-01-21 00:00:00
2021-10-13 00:00:00


In [205]:
usa.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1.0,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1.0,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1.0,0.0
3,2020-01-24,Cook,Illinois,17031.0,1.0,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1.0,0.0


In [206]:
usa[usa['county']=='Snohomish'].tail(10) #cases are cumulative

,date,county,state,fips,cases,deaths
1783964,2021-10-04,Snohomish,Washington,53061.0,59991.0,703.0
1787213,2021-10-05,Snohomish,Washington,53061.0,60219.0,705.0
1790461,2021-10-06,Snohomish,Washington,53061.0,60397.0,710.0
1793710,2021-10-07,Snohomish,Washington,53061.0,60624.0,712.0
1796959,2021-10-08,Snohomish,Washington,53061.0,60795.0,714.0
1800208,2021-10-09,Snohomish,Washington,53061.0,60795.0,714.0
1803457,2021-10-10,Snohomish,Washington,53061.0,60795.0,714.0
1806708,2021-10-11,Snohomish,Washington,53061.0,61336.0,716.0
1809957,2021-10-12,Snohomish,Washington,53061.0,61503.0,716.0
1813206,2021-10-13,Snohomish,Washington,53061.0,61717.0,724.0


Build a new dataset with uniqe county and state. Add the last register closer to the cut date per city. If it is not found (in the sliced dataset), then confirmed cases until that date = 0.

In [207]:
new_USA= usa[['county','state']].drop_duplicates().reset_index(drop=True)
new_USA.to_pickle(saving_path+"/dataframe_empty_USA.pikle")
new_USA.head()

,county,state
0,Snohomish,Washington
1,Cook,Illinois
2,Orange,California
3,Maricopa,Arizona
4,Los Angeles,California


In [208]:
usa_c = usa.copy()
usa_c = usa_c.sort_values('date').drop_duplicates(subset=['county', 'state'] ,keep='last')
usa_c = usa_c.reset_index(drop=True)

In [209]:
usa_c.cases.sum() #44718686.0

44718686.0

In [210]:
usa_c[usa_c.county == 'Unknown'].cases.sum() #No information about county. Drop

181521.0

In [211]:
usa_c = usa_c[usa_c['county'] != 'Unknown']

In [212]:
usa_x = usa_c.groupby(by=['county','state'], as_index=False).sum()
usa_x.head()

,county,state,fips,cases,deaths
0,Abbeville,South Carolina,45001.0,3741.0,49.0
1,Acadia,Louisiana,22001.0,10359.0,248.0
2,Accomack,Virginia,51001.0,3944.0,73.0
3,Ada,Idaho,16001.0,76211.0,708.0
4,Adair,Iowa,19001.0,1038.0,34.0


In [213]:
usa_x.cases.sum() #44,537,165.0 close to official number 46,200,000

44537165.0

In [214]:
usa.cases.sum()

11146541222.0

Population file by county (2020):

In [215]:
usa_pop = pd.read_csv(filepath+'/USA/co-est2020.csv', encoding='cp1252')
usa_pop = usa_pop.loc[:,['STNAME', 'CTYNAME', 'POPESTIMATE2020']] #To get the useful columns
usa_pop = usa_pop.rename(columns = {'STNAME': 'state', 'CTYNAME':'county', 
                                    'POPESTIMATE2020':'population'})
usa_pop = usa_pop[['county', 'state', 'population']]
'''Get rid of the County in the name of the county, to be able to match with the other file'''
usa_pop['county'] = usa_pop['county'].str.replace(r'\ County', '')
mask = usa_pop['county'].str.contains('Parish', regex=True)

'''Municipality and Parish correction of names'''
usa_pop['county'] = np.where(usa_pop['county'].str.contains('Parish', regex=True), 
                                  usa_pop['county'].str.replace(r'\ Parish', ''),
                             usa_pop['county'])
usa_pop['county'] = usa_pop['county'].str.replace('Anchorage Municipality', 'Anchorage')

''''Bristol Bay plus Lake and Peninsula correction of names'''
usa_pop.loc[usa_pop.county == 'Lake and Peninsula Borough', 'county'] = 'Bristol Bay plus Lake and Peninsula'
usa_pop.loc[usa_pop.county == 'Bristol Bay Borough', 'county'] = 'Bristol Bay plus Lake and Peninsula'
new_popul = usa_pop.loc[usa_pop['county'] == 'Bristol Bay plus Lake and Peninsula', 'population'].sum()
usa_pop.loc[usa_pop.county == 'Bristol Bay plus Lake and Peninsula', 'population'] = new_popul

'''Yakutat plus Hoonah-Angoon correction of names'''
usa_pop.loc[usa_pop.county == 'Hoonah-Angoon Census Area', 'county'] = 'Yakutat plus Hoonah-Angoon'
usa_pop.loc[usa_pop.county == 'Yakutat City and Borough', 'county'] = 'Yakutat plus Hoonah-Angoon'
new_popul = usa_pop.loc[usa_pop['county'] == 'Yakutat plus Hoonah-Angoon', 'population'].sum()
usa_pop.loc[usa_pop.county == 'Yakutat plus Hoonah-Angoon', 'population'] = new_popul

'''New York City correction of names'''
usa_pop.loc[(usa_pop.county == 'Bronx') & (usa_pop.state =='New York'), 'county'] = 'New York City'
usa_pop.loc[(usa_pop.county == 'Kings') & (usa_pop.state =='New York'), 'county'] = 'New York City'
usa_pop.loc[(usa_pop.county == 'Queens') & (usa_pop.state =='New York'), 'county'] = 'New York City'
usa_pop.loc[(usa_pop.county == 'Richmond') & (usa_pop.state =='New York'), 'county'] = 'New York City'
new_popul = usa_pop.loc[usa_pop['county'] == 'New York City', 'population'].sum()
usa_pop.loc[usa_pop.county == 'New York City', 'population'] = new_popul

'''Valdez-Cordova Census Area correction of names'''
usa_pop.loc[(usa_pop.county == 'Chugach Census Area') & (usa_pop.state =='Alaska'), 'county'] = 'Valdez-Cordova Census Area'
usa_pop.loc[(usa_pop.county == 'Copper River Census Area') & (usa_pop.state =='Alaska'), 'county'] = 'Valdez-Cordova Census Area'
new_popul = usa_pop.loc[usa_pop['county'] == 'Valdez-Cordova Census Area', 'population'].sum()
usa_pop.loc[usa_pop.county == 'Valdez-Cordova Census Area', 'population'] = new_popul

usa_pop = usa_pop.drop_duplicates().reset_index(drop=True)

/var/folders/2r/dxh5vfq12h7bhlg2f88kpl440000gn/T/ipykernel_24272/1540104892.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  usa_pop['county'] = usa_pop['county'].str.replace(r'\ County', '')
/var/folders/2r/dxh5vfq12h7bhlg2f88kpl440000gn/T/ipykernel_24272/1540104892.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  usa_pop['county'].str.replace(r'\ Parish', ''),


In [216]:
usa_pop['county'][0]

'Alabama'

The two files need to be merged by County and state name (both should match).
In this way, the final merged file will contain localization information, population and cases.

In [217]:
merged_usa = usa_x.merge(usa_pop, how = 'outer', on = ['county', 'state'], copy =False)

In [218]:
merged_usa = merged_usa.drop(merged_usa[merged_usa.state == 'Puerto Rico'].index)
merged_usa = merged_usa.drop(merged_usa[merged_usa.state == 'Virgin Islands'].index)
merged_usa = merged_usa.drop(merged_usa[merged_usa.state == 'Northern Mariana Islands'].index)
merged_usa

,county,state,fips,cases,deaths,population
0,Abbeville,South Carolina,45001.0,3741.0,49.0,24404.0
1,Acadia,Louisiana,22001.0,10359.0,248.0,61918.0
2,Accomack,Virginia,51001.0,3944.0,73.0,32238.0
3,Ada,Idaho,16001.0,76211.0,708.0,494399.0
4,Adair,Iowa,19001.0,1038.0,34.0,7059.0
...,...,...,...,...,...,...
3267,Virginia,Virginia,NaN,NaN,NaN,8590563.0
3268,Washington,Washington,NaN,NaN,NaN,7693612.0
3269,West Virginia,West Virginia,NaN,NaN,NaN,1784787.0
3270,Wisconsin,Wisconsin,NaN,NaN,NaN,5832655.0


If county and state are the same name, the population is the total for that state. We need to get rid of those rows.

In [219]:
merged_usa = merged_usa.drop(merged_usa[merged_usa.county == merged_usa.state].index)

51 cases are from 'Unknown' county. They are going to be dropped.

In [220]:
merged_usa[merged_usa['county']=='Unknown'].shape

(0, 6)

In [221]:
zero_cases = merged_usa[merged_usa['cases'].isna()]

There are no places with zero cases

In [222]:
zero_cases

,county,state,fips,cases,deaths,population


In [223]:
missing_population = merged_usa[merged_usa['population'].isna()]

In [224]:
missing_population

,county,state,fips,cases,deaths,population
1495,Joplin,Missouri,0.0,8626.0,151.0,NaN
1513,Kansas City,Missouri,0.0,62328.0,810.0,NaN


For some reason, the source of information used does not include population for Joplin and Kansas City. It will be added manually using google as source (data from 2019).

* Joplin : 50386 
* Kansas City: 486404

In [225]:
merged_usa.loc[merged_usa.county == 'Joplin', 'population'] =  50386
merged_usa.loc[merged_usa.county == 'Kansas City', 'population'] =  486404

In [226]:
missing_population = merged_usa[merged_usa['population'].isna()]

In [227]:
missing_population

,county,state,fips,cases,deaths,population


In [228]:
merged_usa = merged_usa.drop(columns=['fips', 'deaths'])

In [229]:
merged_usa.to_csv(saving_path+'/USA14Oct.csv' , header=['county', 'state', 'cases', 'population'], 
                 index = False)

In [230]:
merged_usa

,county,state,cases,population
0,Abbeville,South Carolina,3741.0,24404.0
1,Acadia,Louisiana,10359.0,61918.0
2,Accomack,Virginia,3944.0,32238.0
3,Ada,Idaho,76211.0,494399.0
4,Adair,Iowa,1038.0,7059.0
...,...,...,...,...
3222,Yuma,Arizona,41100.0,217824.0
3223,Yuma,Colorado,1165.0,10047.0
3224,Zapata,Texas,2195.0,14172.0
3225,Zavala,Texas,2501.0,11840.0


by State

In [231]:
usa_state = merged_usa[['state','population','cases']]
usa_state = usa_state.reset_index(drop=True)
usa_state = usa_state.groupby(by=['state'], as_index=False).sum()

In [232]:
usa_state.cases.sum()

43999305.0

In [233]:
usa_state

,state,population,cases
0,Alabama,4921532.0,810501.0
1,Alaska,731158.0,124440.0
2,Arizona,7421401.0,1124709.0
3,Arkansas,3013139.0,495970.0
4,California,39368078.0,4815429.0
5,Colorado,5807719.0,702277.0
6,Connecticut,3557006.0,394382.0
7,Delaware,986809.0,138074.0
8,Florida,21733312.0,3601754.0
9,Georgia,10710017.0,1571184.0
